In [1]:
import time
import pandas as pd
import numpy as np
import re
import string

In [2]:
data_w_newlines = pd.read_csv('indeed_data_v2_1000.csv')

#The order and method of data cleaning is influenced by the kind of textual analysis im going to do later.

#Renaming columns and filling NaN values
data_w_newlines = data_w_newlines.rename(columns={"Unnamed: 0": "Job_ID", "job_details": "Job_Details", 'benefits': 'Benefits', 'full_desc': 'Full_Desc','hiring_ins': 'Hiring_Ins'})
data_w_newlines = data_w_newlines.fillna('NA')

#Removing punctuation from benefits, job_details, and full_desc columns
for char in string.punctuation:
    data_w_newlines['Benefits'] = data_w_newlines['Benefits'].str.replace(char, '')
    data_w_newlines['Full_Desc'] = data_w_newlines['Full_Desc'].str.replace(char, '')
    data_w_newlines['Job_Details'] = data_w_newlines['Job_Details'].str.replace(char, '')

#Making all text columns lowercase
data_w_newlines['Benefits'] = data_w_newlines['Benefits'].str.lower()
data_w_newlines['Full_Desc'] = data_w_newlines['Full_Desc'].str.lower()
data_w_newlines['Job_Details'] = data_w_newlines['Job_Details'].str.lower()



data_w_newlines


,Job_ID,Job_Details,Benefits,Full_Desc,Hiring_Ins
0,0,job details\nsalary\n70000 85000 a year\njob ...,benefits\npulled from the full job description...,full job description\nbrainsell is seeking a c...,Hiring Insights\nHiring 1 candidate for this r...
1,1,job details\nsalary\n18 24 an hour\njob type\...,benefits\npulled from the full job description...,we have an opening for a data analyst who is p...,Hiring Insights\nApplication response rate: 90...
2,2,na,na,lake appliance repair is seeking an experience...,Hiring Insights\nJob activity\nPosted 30+ days...
3,3,job details\nsalary\n20 an hour\njob type\nful...,na,parttime data entry analyst solvas financial ...,Hiring Insights\nJob activity\nEmployer review...
4,4,job details\nsalary\n9775 11657 a month\njob ...,benefits\npulled from the full job description...,california public employees’ retirement system...,Hiring Insights\nHiring 1 candidate for this r...
...,...,...,...,...,...
995,995,job details\njob type\nfulltime,benefits\npulled from the full job description...,job description\n\njoin the clean energy revol...,Hiring Insights\nJob activity\nPosted 30+ days...
996,996,job details\nsalary\n70000 110000 a year\njob...,benefits\npulled from the full job description...,job description\ndeliver critical business ins...,Hiring Insights\nJob activity\nPosted 30+ days...
997,997,job details\njob type\nfulltime,na,description\nhealthcare data analyst\nare you ...,Hiring Insights\nJob activity\nPosted 30+ days...
998,998,job details\njob type\nfulltime,na,connectiverx is a leading technologyenabled he...,Hiring Insights\nJob activity\nPosted 8 days ago


In [3]:
#Removing new line special characters, but preserving the new lines in "benefits" column because the /n perfectly seperates benefits
benefits_temp_holder = data_w_newlines['Benefits']

def replace_newline(x):
    if isinstance(x, str):
        return x.replace('\n', ' ')
    return x
data = data_w_newlines.applymap(replace_newline)

data['Benefits'] = benefits_temp_holder


#Create new dataframes for manipulating full_desc and benefits
data_benefits = data[["Job_ID", "Benefits"]].copy()
data_desc = data[['Job_ID', 'Full_Desc']].copy()

#Explode full_desc
data_desc['Full_Desc'] = data_desc['Full_Desc'].str.split()
data_desc = data_desc.explode('Full_Desc')

#Explode benefits
data_benefits['Benefits'] = data_benefits['Benefits'].str.split('\n')
data_benefits = data_benefits.explode('Benefits')

#so now we have the following dataframes: data, data_benefits, data_desc
#lets create a salary column by extracting pay and job type from Job_Details within 'data'

Salary = []
Rate = []
Job_Type = []
for i in range(len(data)):
    match = re.findall(r'\b([\d,]+)\b', data.iloc[i,1])
    Salary.append(match)

for i in range(len(data)): 
    if 'an hour' in data.iloc[i,1]:
        Rate.append('Hourly')
    elif 'a month' in data.iloc[i,1]:
        Rate.append('Monthly')
    elif 'a year' in data.iloc[i,1]:
        Rate.append('Yearly')
    else:
        Rate.append([])
        
for i in range(len(data)):
    if 'Full-time' in data.iloc[i,1]:
        Job_Type.append('Full Time')
    elif 'Part-time' in data.iloc[i,1]:
        Job_Type.append('Part Time')
    elif 'Contract' in data.iloc[i,1]:
        Job_Type.append('Contract')
    else:
        Job_Type.append([])
        
data['Salary Range'] = Salary
data['Pay Rate'] = Rate
data['Job Type'] = Job_Type


Salary_Range_Lower = []
Salary_Range_Upper = []
for i in range(len(data)):
    if len(data.iloc[i,5]) == 2:
        Salary_Range_Lower.append(data.iloc[i,5][0])
        Salary_Range_Upper.append(data.iloc[i,5][1])
    else:
        Salary_Range_Lower.append('NA')
        Salary_Range_Upper.append('NA')

data['Salary Range Lower'] = Salary_Range_Lower
data['Salary Range Upper'] = Salary_Range_Upper

avg = []
for i in range(len(data)):
    if len(data.iloc[i,5]) == 2:
        avg.append((int(data.iloc[i,5][0]) + int(data.iloc[i,5][1]))/2)
    elif len(data.iloc[i,5]) == 1:
        avg.append(int(data.iloc[i,5][0]))
    else:
        avg.append('NA')
data['Salary Average'] = avg

#Dropping Salary Range column because the information is captured in Salary Range Uppper/Lower/Average
data = data.drop(columns = 'Salary Range')


#replacing all empty lists with 'NA'
replace_empty_list = lambda x: "NA" if isinstance(x, list) and not x else x
data = data.applymap(replace_empty_list)
data.head()


,Job_ID,Job_Details,Benefits,Full_Desc,Hiring_Ins,Pay Rate,Job Type,Salary Range Lower,Salary Range Upper,Salary Average
0,0,job details salary 70000 85000 a year job typ...,benefits\npulled from the full job description...,full job description brainsell is seeking a cr...,Hiring Insights Hiring 1 candidate for this ro...,Yearly,NA,70000,85000,77500.0
1,1,job details salary 18 24 an hour job type ful...,benefits\npulled from the full job description...,we have an opening for a data analyst who is p...,Hiring Insights Application response rate: 90%...,Hourly,NA,18,24,21.0
2,2,na,na,lake appliance repair is seeking an experience...,Hiring Insights Job activity Posted 30+ days ago,NA,NA,NA,NA,NA
3,3,job details salary 20 an hour job type fulltime,na,parttime data entry analyst solvas financial ...,Hiring Insights Job activity Employer reviewed...,Hourly,NA,NA,NA,20
4,4,job details salary 9775 11657 a month job typ...,benefits\npulled from the full job description...,california public employees’ retirement system...,Hiring Insights Hiring 1 candidate for this ro...,Monthly,NA,9775,11657,10716.0


In [4]:
#Now lets remove stop words from both benefits and job_description

stop_words = pd.read_csv('stop words.csv').iloc[:,0].tolist()

data_benefits.reset_index(drop=True, inplace=True)
data_benefits = data_benefits[~data_benefits['Benefits'].isin(stop_words)]
data_benefits.reset_index(drop=True, inplace=True)


data_desc.reset_index(drop=True, inplace=True)
data_desc = data_desc[~data_desc['Full_Desc'].isin(stop_words)]
data_desc.reset_index(drop=True, inplace=True)





In [5]:
data.to_csv('Job Postings.csv', sep = '|', index = False)
data_benefits.to_csv('Benefits.csv', sep = '|', index = False, header = False)
data_desc.to_csv('Descriptions.csv', sep = '|', index = False, header = False)